# 3: Detect and align confocal / STED spots, calculate distances

0) Create projections of DNA (target) and RNA (moving) FISH images for easier visualization.
1) Resave each `msr` file to a `tif` file, where each channel is an individual file, with the suffix`_ch{number}`
2) Detect RNA FISH spots, using RS-FISH settings files as input, outputing `merge.csv` list of spots
3) Detect DNA FISH spots images, using RS-FISH settings files as input, outputing `merge.csv` list of spots
4) Get global coordinates for DNA and RNA FISH spots
5) Align RNA to DNA FISH spots based on bead alignment from notebook 2)
6) Match DNA and RNA spots and calculate distances

For more details about required in and output please look at the individual notebooks used in the pipeline.

In [ ]:
import os
import queue
import pandas as pd
from pathlib import Path
from datetime import datetime
import papermill as pm
from multiprocessing import Pool
import concurrent.futures

def run_notebook(parameters,notebook_to_run,parameters_common={}):
    
    # change to directory where the notebook is (resolve relative imports)
    os.chdir(Path(notebook_to_run).absolute().parent)
    
    # run notebook
    for parameters_spec in parameters_list:
        parameters = {**parameters_common, **parameters_spec}

        pm.execute_notebook(
           notebook_to_run,
           '/dev/null',
           parameters=parameters)

# 1) Spot detection RNA
**In:** 
- `tif_subfolder` containing single channel `tif` images
- `out_subfolder` containing spot detection settings file `chX.txt` generated via RS-FISH Fiji plugin for each channel specified in `channels`
- `acquisition_info.json` file with the sample metadata which you wish to be added to your spots

**Out:**
- folder containing a `csv` file for each provioded `tif`
- `merge.csv` file combining spots and their metadata from all images in the batch
- vis folder with detection visualization for all `tif`s as `png` (good for checking detection results)

In [ ]:
parameters_list = [
    {"in_path": "/home/stumberger/ep2024/RNA_DNA_FISH_spot_detection/example/RNAFISH/"}
]

parameters_common = {"channels": [0],
                    "out_subfolder": "detections_confocal/"}

notebook_to_run = "/home/stumberger/ep2024/subscripts/RS-FISH_spot_detection.ipynb"

run_notebook(parameters_list,notebook_to_run,parameters_common)

# 2) Spot detection DNA
See 1)

In [ ]:
parameters_list = [
    {"in_path": "/home/stumberger/ep2024/RNA_DNA_FISH_spot_detection/example/DNAFISH/"}
]

parameters_common = {"channels": [0,1],
                    "out_subfolder": "detections_confocal/"}

notebook_to_run = "/home/stumberger/image-analyis-recipes/spot-detection/RS-FISH_spot_detection.ipynb"

run_notebook(parameters_list,notebook_to_run,parameters_common)

# 3) Get global coordinates

In [ ]:
parameters_list = [
    {"in_path": "/home/stumberger/ep2024/RNA_DNA_FISH_spot_detection/example/DNAFISH/"},
    {"in_path": "/home/stumberger/ep2024/RNA_DNA_FISH_spot_detection/example/RNAFISH/"}
]

### common parameters
parameters_common = {
    "raw_subpath" : "raw", # subpath where raw h5/msr images are saved
    "spots_subpath": "detections_confocal/merge.csv", # single or multiple (*) spot files
    "out_subpath" : "detections_confocal/", #w here to save results
    
    "coordinate_column_names" : ['z', 'y', 'x'], # naming of xyz coordinates
    "global_coordinate_column_names" : ['z_global_um', 'y_global_um', 'x_global_um'], # how to call global coordinate columns
    "image_file_column_name" : 'img', # name of img file column
    
    "use_h5_for_metadata" : False, # use h5 (faster) or msr? 
}

notebook_to_run = "/home/stumberger/ep2024/subscripts/get_global_coordinates_sted.ipynb"

run_notebook(parameters_list,notebook_to_run,parameters_common)

# 4) Align RNA and DNA FISH spots based on beads alignment

In [ ]:
parameters_list = [
    {"base_path_target": "/home/stumberger/ep2024/RNA_DNA_FISH_spot_detection/example/DNAFISH/",
     "base_path_moving": "/home/stumberger/ep2024/RNA_DNA_FISH_spot_detection/example/RNAFISH/"}
]

### common parameters
parameters_common = {
    "df_subdir_moving": "detections_confocal/merge_global_coords.csv",
    "df_subdir_target": "detections_confocal/merge_global_coords.csv",
    "alignment_params_file_moving": "alignment_parameters/alignment_parameters_global.json",
    "out_subdir" : "detections_confocal",
    "save_projections" : True,
    "projections_subdir" : "vis"
}

notebook_to_run = "/home/stumberger/ep2024/subscripts/apply_alignment_to_coordinates.ipynb"

run_notebook(parameters_list,notebook_to_run,parameters_common)

# 5) Calculate DNA-DNA and RNA-DNA distances

In [ ]:
parameters_list = [
    {"in_path1": "/home/stumberger/ep2024/RNA_DNA_FISH_spot_detection/example/DNAFISH/", # DNA FISH path
     "in_path2": "/home/stumberger/ep2024/RNA_DNA_FISH_spot_detection/example/RNAFISH/", # RNA FISH path 
     "out": "/home/stumberger/ep2024/RNA_DNA_FISH_spot_detection/example/distances_confocal.csv" } # which directory to save the output to 
]

### common parameters
parameters_common = {
    "channels": [0,1], # which DNA channels to match
    "ch_p": 0 # which channel is the promoter?
}

notebook_to_run = "/home/stumberger/ep2024/subscripts/paired_spot_distances_2_channels_conf-RNA_DNA.ipynb"

run_notebook(parameters_list,notebook_to_run,parameters_common)